In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import proj_root, os
os.chdir(proj_root())

In [8]:
bad_code = """
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))
"""
from spot.experiments.type4py import cst, remove_newer_syntax

bad_code = remove_newer_syntax(cst.parse_module(bad_code)).code
print(bad_code)



from typing import Any, List, Tuple, Dict, Set, Union, Optional
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))



In [9]:
import requests
from spot.utils import proj_root, read_file

bad_code = """
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))
"""

r = requests.post("https://type4py.com/api/predict?tc=0", bad_code)
r.json()


{'error': None,
 'response': {'classes': [],
  'funcs': [{'docstring': {'func': None, 'long_descr': None, 'ret': None},
    'fn_lc': [[2, 0], [9, 32]],
    'fn_var_ln': {'segs': [[3, 4], [3, 8]], 'tree': [[6, 8], [6, 12]]},
    'fn_var_occur': {'segs': [['segs', 'append', 'tree', 'attr'],
      ['segs', 'append', 'tree', 'id'],
      ['tuple', 'reversed', 'segs']],
     'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'name': 'parse_qualified_name',
    'params': {'tree': 'ast.Attribute'},
    'params_descr': {'tree': ''},
    'params_occur': {'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'params_p': {'args': [], 'kwargs': [], 'tree': []},
    'q_na

In [3]:
from spot.experiments.type4py import Type4PyResponseParser
from spot.static_analysis import FunctionSignature

pred_signatures = Type4PyResponseParser("spot.model").parse(r.json())
for path, sig in pred_signatures.items():
    if isinstance(sig, FunctionSignature):
        print(path)
        print("\t" + str(sig))


spot.model/dynamic_dataloader
	FuncSig((collate_fn: cst'torch.Tensor', dataset: cst'torch.Tensor', max_tokens: cst'int', shuffle: cst'Optional[bool]') -> cst'str')
spot.model/DecodingArgs.scale_ctx_size
	MethodSig((factor: cst'float') -> cst'List[str]')
spot.model/DecodingArgs.__repr__
	MethodSig(() -> cst'str')
spot.model/DatasetPredResult.accuracies
	MethodSig((common_type_names: cst'str') -> cst'str')
spot.model/DatasetPredResult.group_by_repo
	MethodSig(() -> cst'dict')
spot.model/ModelWrapper.scale_ctx_size
	MethodSig(() -> cst'str')
spot.model/ModelWrapper.decode_row
	MethodSig((n_labels: cst'int', row: cst'str') -> cst'str')
spot.model/ModelWrapper.predict_on_batch
	MethodSig((batch: cst'bool', num_return_sequences: cst'Callable[..., T]') -> cst'str')
spot.model/ModelWrapper.predict
	MethodSig((dataset: cst'numpy.ndarray', num_return_sequences: cst'int', tqdm_args: cst'bytes') -> cst'str')
spot.model/ModelWrapper.save_pretrained
	MethodSig((path: cst'List[str]') -> MISSING)
spot

In [6]:
from spot.static_analysis import PythonProject, SignatureErrorAnalysis, AccuracyMetric
from spot.visualization import pretty_print_dict, assert_eq

ex_proj = PythonProject.from_root(
    proj_root(), # file_filter=lambda f: f.name == "model.py"
)
label_signatures = {e.path: e.get_signature() for e in ex_proj.all_elems()}

ubiq_names = {"str", "int", "list", "bool", "float"}
acc_metric = AccuracyMetric(common_type_names=ubiq_names)
analysis = SignatureErrorAnalysis(
    {"spot": pred_signatures}, {"spot": label_signatures}, acc_metric,
)
pretty_print_dict(analysis.accuracies)


NameError: name 'pred_signatures' is not defined

In [4]:
from spot.static_analysis import PythonProject
from spot.utils import *
from spot.visualization import pretty_print_dict, assert_eq
from spot.experiments.type4py import eval_type4py_on_projects
from spot.function_dataset import data_project_from_dir


dataset_name = "ManyTypes4Py"

# test_projects = [PythonProject.from_root(proj_root(), ignore_dirs={".venv", "data"})]

repos_dir = get_dataset_dir(dataset_name) / "repos" / "test"
test_repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()][:5]
test_projects = pmap(
    data_project_from_dir,
    test_repo_paths,
    desc="Loading test projects",
)
eval_r = eval_type4py_on_projects(test_projects, max_workers=4)


Calling Type4Py: 100%|██████████| 93/93 [00:37<00:00,  2.49it/s]


In [7]:
from spot.static_analysis import SignatureErrorAnalysis, AccuracyMetric

ubiq_names = {"str", "int", "list", "bool", "float"}
# acc_metric = AccuracyMetric(common_type_names=ubiq_names, match_base_only=True, name="base_acc")
acc_metric = AccuracyMetric(common_type_names=ubiq_names)
analysis = SignatureErrorAnalysis(
    eval_r.pred_maps, eval_r.label_maps, acc_metric, error_on_mismatched_signature=False
)
pretty_print_dict(analysis.accuracies)


acc: 32.05% (count=964)
acc_by_common:
   rare: 2.28% (count=569)
   common: 74.94% (count=395)
acc_by_cat:
   FuncArg: 32.14% (count=448)
   FuncReturn: 32.65% (count=392)
   ClassAtribute: 29.51% (count=122)
   GlobalVar: 50.00% (count=2)
acc_label_size: 1.2521
acc_pred_size: 1.111
n_label_types: 1211
n_skipped_types: 0
n_missing_types: 11
